<a href="https://colab.research.google.com/github/kr7/IntelligensModszerekTantargy/blob/main/Idosorok_osztalyozasa_neuralis_haloval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from ipywidgets import IntProgress
from IPython.display import display

from sklearn.metrics import roc_auc_score

In [2]:
!wget http://www.timeseriesclassification.com/aeon-toolkit/Archives/Univariate2018_arff.zip

--2024-04-10 13:22:10--  http://www.timeseriesclassification.com/aeon-toolkit/Archives/Univariate2018_arff.zip
Resolving www.timeseriesclassification.com (www.timeseriesclassification.com)... 109.123.71.232
Connecting to www.timeseriesclassification.com (www.timeseriesclassification.com)|109.123.71.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548832105 (523M) [application/zip]
Saving to: ‘Univariate2018_arff.zip’

Univariate2018_arff 100%[===================>] 523.41M  30.5MB/s    in 18s     

2024-04-10 13:22:28 (29.0 MB/s) - ‘Univariate2018_arff.zip’ saved [548832105/548832105]



In [3]:
!unzip Univariate2018_arff.zip

Archive:  Univariate2018_arff.zip
   creating: Univariate_arff/
   creating: Univariate_arff/ACSF1/
  inflating: Univariate_arff/ACSF1/ACSF1.txt  
  inflating: Univariate_arff/ACSF1/ACSF1_TEST.arff  
  inflating: Univariate_arff/ACSF1/ACSF1_TEST.txt  
  inflating: Univariate_arff/ACSF1/ACSF1_TRAIN.arff  
  inflating: Univariate_arff/ACSF1/ACSF1_TRAIN.txt  
  inflating: Univariate_arff/ACSF1/README.md  
   creating: Univariate_arff/Adiac/
  inflating: Univariate_arff/Adiac/Adiac.txt  
  inflating: Univariate_arff/Adiac/Adiac_TEST.arff  
  inflating: Univariate_arff/Adiac/Adiac_TEST.txt  
  inflating: Univariate_arff/Adiac/Adiac_TRAIN.arff  
  inflating: Univariate_arff/Adiac/Adiac_TRAIN.txt  
   creating: Univariate_arff/AllGestureWiimoteX/
  inflating: Univariate_arff/AllGestureWiimoteX/AllGestureWiimoteX.txt  
  inflating: Univariate_arff/AllGestureWiimoteX/AllGestureWiimoteX_TEST.arff  
  inflating: Univariate_arff/AllGestureWiimoteX/AllGestureWiimoteX_TEST.txt  
  inflating: Univari

In [4]:
train_data_with_class_labels = np.genfromtxt('Univariate_arff/ECG200/ECG200_TRAIN.txt')
test_data_with_class_labels = np.genfromtxt('Univariate_arff/ECG200/ECG200_TEST.txt')

train_data = train_data_with_class_labels[:,1:]
test_data = test_data_with_class_labels[:,1:]

train_labels = train_data_with_class_labels[:,0]
train_labels[train_labels==-1] = 0
test_labels = test_data_with_class_labels[:,0]
test_labels[test_labels==-1] = 0

NUM_CLASSES = 2
NUM_INPUT_FEATURES = len(train_data[0])

In [5]:
def convolution(in_channels, out_channels, kernel_size, padding='same'):
    return nn.Conv1d(in_channels=in_channels, out_channels=out_channels,
                kernel_size=kernel_size, padding=padding, stride=1)


class FCN(nn.Module): # use this class for experiments with FCN
    def __init__(self):
        super(FCN, self).__init__()

        self.conv1 = convolution(1, 128, 8, 0)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = convolution(128, 256, 5, 0)
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = convolution(256, 128, 3, 0)
        self.bn3 = nn.BatchNorm1d(128)

        self.out = nn.Linear(128, 2)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)

        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)

        # Global Average Pooling
        x = torch.mean(x, 2)
        x = x.view(-1, 128)

        x = self.out(x)
        x = self.softmax(x)

        return x


class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()

        self.conv11 = convolution(1, 64, 8)
        self.bn11 = nn.BatchNorm1d(64)
        self.conv12 = convolution(64, 64, 5)
        self.bn12 = nn.BatchNorm1d(64)
        self.conv13 = convolution(64, 64, 3)
        self.bn13 = nn.BatchNorm1d(64)

        self.conv21 = convolution(64, 128, 8)
        self.bn21 = nn.BatchNorm1d(128)
        self.conv22 = convolution(128, 128, 5)
        self.bn22 = nn.BatchNorm1d(128)
        self.conv23 = convolution(128, 128, 3)
        self.bn23 = nn.BatchNorm1d(128)

        self.conv31 = convolution(128, 128, 8)
        self.bn31 = nn.BatchNorm1d(128)
        self.conv32 = convolution(128, 128, 5)
        self.bn32 = nn.BatchNorm1d(128)
        self.conv33 = convolution(128, 128, 3)
        self.bn33 = nn.BatchNorm1d(128)

        self.out = nn.Linear(128, NUM_CLASSES)
        self.softmax = torch.nn.Softmax(dim=1)


    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)

        x1 = x
        x = self.conv11(x)
        x = self.bn11(x)
        x = torch.relu(x)
        x = self.conv12(x)
        x = self.bn12(x)
        x = torch.relu(x)
        x = self.conv13(x)
        x = self.bn13(x)
        #x = torch.relu(x)
        x = x + x1
        x = torch.relu(x)

        x1 = x
        x = self.conv21(x)
        x = self.bn21(x)
        x = torch.relu(x)
        x = self.conv22(x)
        x = self.bn22(x)
        x = torch.relu(x)
        x = self.conv23(x)
        x = self.bn23(x)
        x = torch.relu(x)

        x1 = torch.cat( (x1,x1), 1)
        x = x + x1
        x = torch.relu(x)

        x1 = x
        x = self.conv31(x)
        x = self.bn31(x)
        x = torch.relu(x)
        x = self.conv32(x)
        x = self.bn32(x)
        x = torch.relu(x)
        x = self.conv33(x)
        x = self.bn33(x)
        #x = torch.relu(x)
        x = x + x1
        x = torch.relu(x)

        # Global Average Pooling
        x = torch.mean(x, 2)
        x = x.view(-1, 128)

        x = self.out(x)
        x = self.softmax(x)

        return x

In [15]:
batch_size = 16
epochs = 1000
lr = 1e-5

epoch_bar= IntProgress(min=0, max=epochs)
display(epoch_bar)

train_dataset = torch.utils.data.TensorDataset(torch.Tensor(train_data),
                torch.Tensor(torch.nn.functional.one_hot(torch.LongTensor(train_labels))).float() )
trainloader = torch.utils.data.DataLoader( train_dataset, batch_size=batch_size, shuffle=True)

#net = FCN()
net = ResNet()

#net.cuda() # - ha van GPU

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

for epoch in range(epochs):
  epoch_bar.value = epoch
  for inputs, targets in trainloader:
    #inputs = inputs.cuda() # - ha van GPU
    #targets = targets.cuda() # - ha van GPU

    optimizer.zero_grad()
    predictions = net(inputs)

    loss = criterion(predictions, targets)
    loss.backward()
    optimizer.step()

IntProgress(value=0, max=1000)

In [16]:
test_dataset = torch.utils.data.TensorDataset(
    torch.Tensor(test_data),
    torch.LongTensor(test_labels) )
testloader = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size)


correct = 0
total = 0

for inputs, targets in testloader:
  #inputs = inputs.cuda() # - ha van GPU
  #targets = targets.cuda() # - ha van GPU

  with torch.no_grad():
    output = net(inputs)
    _, predictions = torch.max(output.data, dim=1)
    total += targets.cpu().size(0)
    correct += (predictions == targets).sum().cpu().item()


In [17]:
correct

88